In [1]:
import pandas as pd
import os
import re


In [4]:
# está función la vamos a utilizar para coger todos los archivos que vayamos descargando en nuestra carpeta de excels
# donde el que tiene mayor índice en el nombre es el archivo más reciente. A veces con bs4 no coge un dato y devuelve none
# entonces aquí con pandas lo que vamos a hacer es crear un excel con los datos más actualizados y así nos aseguramos que
# si hay algún dato en algún momento que bs4 no se ha descargado mantenemos el antiguo

# Esta función es para los equipos nba

def obtener_archivos_ordenados(path, prefijo):
    archivos = [archivo for archivo in os.listdir(path) if archivo.startswith(prefijo) and archivo.endswith('.xlsx')]
    archivos_ordenados = sorted(archivos, key=lambda x: int(re.search(r'(\d+)', x.replace(prefijo, '').replace('.xlsx', '')).group(1) if re.search(r'(\d+)', x) else 0))
    archivos_ordenados = [os.path.join(path, archivo) for archivo in archivos_ordenados]
    return archivos_ordenados

def actualizar_datos_nba_modificado(archivos):
    df_final = pd.DataFrame()

    for archivo in archivos:
        df_actual = pd.read_excel(archivo)
        df_actual.replace('none', pd.NA, inplace=True)

        if not df_final.empty:
            for columna in df_actual.columns:
                if columna != 'ID Equipo':
                    df_final = pd.merge(df_final, df_actual[['ID Equipo', columna]], on='ID Equipo', suffixes=('', '_actual'))
                    df_final[columna] = df_final.apply(
                        lambda fila: fila[columna] if pd.isna(fila[columna + '_actual']) else fila[columna + '_actual'], axis=1)
                    df_final.drop(columna + '_actual', axis=1, inplace=True)
        else:
            df_final = df_actual

    return df_final

# Ruta de donde coge el directorio para   los archivos Excel
path = 'excels/'
prefijo_archivo = 'equipos_nba'

# Obtener la lista de archivos ordenados
archivos_ordenados = obtener_archivos_ordenados(path, prefijo_archivo)

df_equipos_nba_actualizado = actualizar_datos_nba_modificado(archivos_ordenados)

# Creamos el directorio si no existe y ya generamos el archivo en excel con los datos actualizados
path_actualizado = 'excels/actualizados/'
if not os.path.exists(path_actualizado):
    os.makedirs(path_actualizado)

nombre_archivo = 'equipos_nba_actualizado.xlsx'
df_equipos_nba_actualizado.to_excel(path_actualizado + nombre_archivo, index=False)

In [3]:
# Esta función es para estadistica jugadores nba

def obtener_archivos_ordenados(path, prefijo):
    archivos = [archivo for archivo in os.listdir(path) if archivo.startswith(prefijo) and archivo.endswith('.xlsx')]
    archivos_ordenados = sorted(archivos, key=lambda x: int(re.search(r'(\d+)', x.replace(prefijo, '').replace('.xlsx', '')).group(1) if re.search(r'(\d+)', x) else 0))
    archivos_ordenados = [os.path.join(path, archivo) for archivo in archivos_ordenados]
    return archivos_ordenados

def actualizar_datos_estadisticas(archivos):
    df_final = pd.DataFrame()

    for archivo in archivos:
        df_actual = pd.read_excel(archivo)
        df_actual.rename(columns={'#': 'id_jugador', 'TEAM': 'ID Equipo'}, inplace=True)
        df_actual.replace('none', pd.NA, inplace=True)

        if not df_final.empty:
            for columna in df_actual.columns:
                if columna != 'id_jugador':
                    df_final = pd.merge(df_final, df_actual[['id_jugador', columna]], on='id_jugador', suffixes=('', '_actual'))
                    df_final[columna] = df_final.apply(
                        lambda fila: fila[columna] if pd.isna(fila[columna + '_actual']) else fila[columna + '_actual'], axis=1)
                    df_final.drop(columna + '_actual', axis=1, inplace=True)
        else:
            df_final = df_actual

    return df_final

# Ruta de donde coge el directorio para los archivos Excel
path = 'excels/'
prefijo_archivo = 'estadisticas_puntos_jugadores'

# Obtener la lista de archivos ordenados
archivos_ordenados = obtener_archivos_ordenados(path, prefijo_archivo)

# Actualizar los datos de las estadísticas
df_estadisticas_actualizado = actualizar_datos_estadisticas(archivos_ordenados)

# Crear directorio si no existe y guardar el DataFrame actualizado
path_actualizado = 'excels/actualizados/'
if not os.path.exists(path_actualizado):
    os.makedirs(path_actualizado)

# Guardar el DataFrame actualizado en un nuevo archivo Excel
nombre_archivo = 'estadistica_jugadores_nba_actualizado.xlsx'
df_estadisticas_actualizado.to_excel(path_actualizado + nombre_archivo, index=False)